In [1]:
import numpy as np
import pandas as pd
import joblib
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import  OneHotEncoder
from category_encoders import BinaryEncoder
from sklearn import preprocessing
from sklearn.metrics import classification_report
from zipfile import ZipFile 
import requests
import pickle

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


ModuleNotFoundError: ignored

## Load test data and label

In [0]:
test_data = pd.read_csv('orange_small_test.data', sep = '\t',header=None)
test_appetency = pd.read_csv('orange_small_train_appetency_test.labels',header=None).astype('float')
test_churn = pd.read_csv('orange_small_train_churn_test.labels',header=None).astype('float')
test_upselling = pd.read_csv('orange_small_train_upselling_test.labels',header=None).astype('float')

test_appetency.columns = ['appetency']
test_churn.columns = ['churn']
test_upselling.columns = ['upselling']
test_churn = (test_churn + 1)/2
test_upselling = (test_upselling + 1)/2
test_appetency = (test_appetency + 1)/2

In [0]:
#train_info_dic = {}
with open('train_info_dic.pkl','rb') as f:
    train_info_dic=pickle.load(f,encoding='bytes')

test_data.columns = train_info_dic['orig_columns']
test_data_drop_na = test_data.drop(train_info_dic['drop_list'],axis=1)
test_data_light_gbm = test_data_drop_na.copy()
category_columns = test_data_drop_na.select_dtypes(['object']).columns
test_data_light_gbm[category_columns] = test_data_drop_na[category_columns].apply(lambda col: col.astype('category'))



In [0]:
category_columns = train_info_dic['category_columns']
test_data_drop_na[category_columns] = test_data_drop_na[category_columns].apply(lambda col: col.astype('category'))

test_data_drna_mean = test_data_drop_na.fillna(train_info_dic['train_data_mean'])

test_data_drna_mean_scale = test_data_drna_mean
numberic_columns_list = train_info_dic['numberic_columns']  
std_scaler = train_info_dic['std_scaler'] 
test_data_drna_mean_scale[numberic_columns_list] = std_scaler.transform(test_data_drna_mean[numberic_columns_list])

binary_encoder = train_info_dic['binary_encoder']
test_data_drna_mean_scale_bin = binary_encoder.transform(test_data_drna_mean_scale)

In [0]:
'''
imp = train_info_dic['imp']
numberic_columns = test_data_drop_na.select_dtypes(exclude=['category']).columns
test_data_drop_na_imputed = imp.transform(test_data_drop_na[numberic_columns])
test_data_drop_na[numberic_columns] = pd.DataFrame(test_data_drop_na_imputed)
test_data_drna_mean = test_data_drop_na
'''

"\nimp = train_info_dic['imp']\nnumberic_columns = test_data_drop_na.select_dtypes(exclude=['category']).columns\ntest_data_drop_na_imputed = imp.transform(test_data_drop_na[numberic_columns])\ntest_data_drop_na[numberic_columns] = pd.DataFrame(test_data_drop_na_imputed)\ntest_data_drna_mean = test_data_drop_na\n"

In [0]:
model_rf_appetency = joblib.load("random_forest_appetency.model")
result_rf_appetency = model_rf_appetency.predict(test_data_drna_mean_scale_bin)
print(classification_report(test_appetency,result_rf_appetency))

              precision    recall  f1-score   support

         0.0       0.99      0.96      0.97     24548
         1.0       0.11      0.25      0.15       452

    accuracy                           0.95     25000
   macro avg       0.55      0.61      0.56     25000
weighted avg       0.97      0.95      0.96     25000



In [0]:

model_rf_churn = joblib.load("random_forest_churn.model")
result_rf_churn = model_rf_churn.predict(test_data_drna_mean_scale_bin)
print(classification_report(test_churn,result_rf_churn))

              precision    recall  f1-score   support

         0.0       0.95      0.80      0.87     23165
         1.0       0.16      0.46      0.24      1835

    accuracy                           0.78     25000
   macro avg       0.55      0.63      0.55     25000
weighted avg       0.89      0.78      0.82     25000



In [0]:
model_rf_upselling = joblib.load("random_forest_upselling.model")
result_rf_upselling = model_rf_upselling.predict(test_data_drna_mean_scale_bin)
print(classification_report(test_upselling,result_rf_upselling))

              precision    recall  f1-score   support

         0.0       0.96      0.99      0.97     23181
         1.0       0.77      0.47      0.58      1819

    accuracy                           0.95     25000
   macro avg       0.86      0.73      0.78     25000
weighted avg       0.95      0.95      0.95     25000



In [0]:
model_lightgbm_appetency = joblib.load("light_gbm_appetency.model")
result_lgb_appetency = model_lightgbm_appetency.predict(test_data_light_gbm)
print(classification_report(test_appetency,result_lgb_appetency))

              precision    recall  f1-score   support

         0.0       0.99      0.97      0.98     24548
         1.0       0.12      0.23      0.16       452

    accuracy                           0.95     25000
   macro avg       0.55      0.60      0.57     25000
weighted avg       0.97      0.95      0.96     25000



In [0]:
model_lightgbm_churn = joblib.load("light_gbm_churn.model")
result_lgb_churn = model_lightgbm_churn.predict(test_data_light_gbm)
print(classification_report(test_churn,result_lgb_churn))

              precision    recall  f1-score   support

         0.0       0.94      0.94      0.94     23165
         1.0       0.21      0.20      0.20      1835

    accuracy                           0.88     25000
   macro avg       0.57      0.57      0.57     25000
weighted avg       0.88      0.88      0.88     25000



In [0]:
model_lightgbm_upselling = joblib.load("light_gbm_upselling.model")
result_lgb_upselling = model_lightgbm_upselling.predict(test_data_light_gbm)
print(classification_report(test_upselling,result_lgb_upselling))

              precision    recall  f1-score   support

         0.0       0.93      0.97      0.95     23165
         1.0       0.24      0.12      0.16      1835

    accuracy                           0.91     25000
   macro avg       0.59      0.54      0.55     25000
weighted avg       0.88      0.91      0.89     25000

              precision    recall  f1-score   support

         0.0       0.96      0.98      0.97     23181
         1.0       0.62      0.48      0.54      1819

    accuracy                           0.94     25000
   macro avg       0.79      0.73      0.75     25000
weighted avg       0.93      0.94      0.94     25000



In [0]:
result_new = result_rf + result_lgb

In [0]:
result_new[result_new==2].shape

In [0]:
result_new[result_new==1].shape

In [0]:
def covert_result(x):
    if x>=2:
        x=1
    else:
        x=0
    return x

In [0]:
vec_covert_result = np.vectorize(covert_result)

In [0]:
vec_result = vec_covert_result(result_new)

In [0]:
print(classification_report(test_appetency,vec_result))